# Collapsing traceroutes
this code
a) looks up all the unique traceroutes (according to hash).
b) prepends source site IP
c) For each IP (not first or last) make a list of all previous and following IPs (and how often it happened)
find all IPs that had matching lists.
d) makes list of all the IPs, make a list of IP equivalences. 


In [1]:
%matplotlib inline
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from datetime import datetime
import math
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import gridspec
from datetime import datetime
import numpy as np
import pandas as pd

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)
my_index = "network_weather_2-2016.9.*"

my_query = {}

##  Select your link

In [2]:
# CHANGE SO IT RETREIVES ONLY UNIQUE hashes.

# sS='UC'
# srcSiteOWDServer = "192.170.227.160"
# srcSiteThroughputServer = "192.170.227.162"

sS='CERN-PROD'
srcSiteOWDServer = "128.142.223.247"
srcSiteThroughputServer = "128.142.223.246"

# dS='IU'
# destSiteOWDServer = "149.165.225.223"
# destSiteThroughputServer = "149.165.225.224"

# dS='UIUC'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.96.9"

# dS='ICCN'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.126.132"

dS='pic'
destSiteOWDServer = "193.109.172.188"
destSiteThroughputServer = "193.109.172.187"

## Get the data from Elasticsearch, and store the record based on its type

In [3]:
my_query = {
    "size":1,
    "_source": {
        "include": [ 'src','dest','hops','hash' ]
    },
    'query':{
        'filtered':{
            'query': { "match_all": {} },
            'filter':{
                'bool':{
                    'must':[
                        {'range': {'timestamp': {'gte': '2016-09-15', 'lt': '2016-09-30'}}},
                        {'term': {'_type': 'traceroute'}},
#                         {'bool':
#                             {'should':[
#                                 {'term': {'src': srcSiteOWDServer}},
#                                 {'term': {'src': srcSiteThroughputServer}},
#                                 {'term': {'src': destSiteOWDServer}},
#                                 {'term': {'src': destSiteThroughputServer}}
#                             ]}
#                         }
#                         ,
#                         {'bool':
#                             {'should':[
#                                 {'term': {'dest': destSiteOWDServer}},
#                                 {'term': {'dest': destSiteThroughputServer}},
#                                 {'term': {'dest': srcSiteOWDServer}},
#                                 {'term': {'dest': srcSiteThroughputServer}}
#                             ]}
#                         }
                    ]
                }
            }
        }
    },
    "aggs": {
        "grouped_by_hash": {
          "terms": { 
                "field": "hash",
                "size":0
            },
          "aggs": {
              "top_hash_hits": {
                  "top_hits": {
                      "sort": [ { "_score": { "order": "desc" } } ],
                      "size": 1
                  }
              }
          }
       }
    }
}

results = es.search(body=my_query, index=my_index, request_timeout=12000)
print(len(results['aggregations']['grouped_by_hash']['buckets']))

buckets=results['aggregations']['grouped_by_hash']['buckets']
results=[]
# c=0
for b in buckets:
#     if c==0: 
#         print(b)
#         c+=1
    if b['doc_count']>20: # we don't want very short jumps
        results.append(b['top_hash_hits']['hits']['hits'][0])

186551


In [5]:
AllIPs=set()
uniquePaths={}
count = 0
for res in results:
    if count<1: print(res)
    count += 1
    # for data before Oct. 15 2016
    s=res['_source']['src']
    d=res['_source']['dest']
    hops=[s]
    hops += res['_source']['hops']
    h=res['_source']['hash']
    if h not in uniquePaths: 
        if hops[-1]==None: hops[-1]=d
        uniquePaths[h] = hops
        for h in hops:
            if h!=None:
                AllIPs.add(h)
print('uniquePaths:',len(uniquePaths))
print('AllIPs:', len(AllIPs))

{'_type': 'traceroute', '_id': 'AVcsnxJ5sPCefqD7eNvA', '_index': 'network_weather_2-2016.9.15', '_score': 1.0, '_source': {'hash': 8510477699910703797, 'src': '149.165.225.2', 'destVO': 'ATLAS', 'rtts': [11.05, 0.312, 5.222, 5.713, 5.239], 'timestamp': 1473917822000, 'hops': ['149.165.225.254', '149.165.254.249', '149.165.227.22', '192.170.224.78', '192.170.227.162'], 'destSite': 'MWT2', 'dest': '192.170.227.162', 'MA': '192.170.227.162', 'destProduction': True, 'ttls': [1, 2, 3, 4, 5], 'srcProduction': False}}
docs: 18266
uniquePaths: 18266
AllIPs: 3536


### Look for one off differences

In [7]:
AllSegs={}
for uP in uniquePaths.values():
    l=len(uP)
    #print(uP, l)
    for si in range(1,l-1):
        if uP[si]==None: continue
        if uP[si] not in AllSegs: AllSegs[uP[si]]=set()
        AllSegs[uP[si]].add(uP[si-1])
        AllSegs[uP[si]].add(uP[si+1])
print("non-edge devices:", len(AllSegs))

non-edge devices: 3171


In [15]:
simpleIPs=set()
toLook=0
for s in AllSegs:
    if len(AllSegs[s])<3:
        simpleIPs.add(s)
        simpleIPs=simpleIPs.union(AllSegs[s])
    else: toLook+=1
    if len(AllSegs[s])>2: 
        print(s, AllSegs[s])
print ('simple:',len(simpleIPs) , 'toLook:', toLook)

164.58.244.46 {'164.58.244.15', '164.58.244.56', '198.129.77.54', '164.58.244.17', '164.58.7.49'}
2001:798:111:1::2 {'2001:798:111:1::2', '2001:798:111:1::1d', None, '2001:660:5009::5:2', '2001:798:111:1::61', '2001:660:7904:117:1::1'}
117.103.111.158 {'134.55.42.42', '117.103.111.206', '134.55.218.190'}
2001:48a8:48ff:ff0c::2 {None, '2001:504:0:4::237:1', '2001:48a8:5fff:13::2'}
193.149.1.17 {'130.206.245.1', '130.206.245.9', '130.206.212.17', '87.245.233.30', '130.206.245.37', '87.245.233.22'}
192.76.22.1 {'163.1.5.210', None, '163.1.120.78', '163.1.5.211'}
192.70.69.142 {'192.70.69.129', '141.142.128.18', '141.142.128.10'}
159.226.254.118 {'159.226.253.42', '192.168.1.25', '159.226.253.50'}
159.93.161.1 {'159.93.2.253', '159.93.229.150', '194.190.165.254'}
146.97.33.2 {'62.40.124.198', '146.97.37.250', '146.97.35.234', '146.97.37.202', '212.187.139.166', '146.97.33.6', '146.97.40.254', '146.97.33.22', '146.97.37.194'}
129.143.59.42 {'129.143.19.122', '129.143.59.38', '129.143.57.62'

In [9]:
#print(EquivalentDevices)
groups=[]
for s in EquivalentDevices:
    for d in EquivalentDevices[s]:
        #print (s,d,EquivalentDevices[s][d])
        groups.append(set(EquivalentDevices[s][d]))

In [10]:
print(groups)

[{'2001:660:7903:204:2::2', '2001:660:7904:121:2::2'}, {'159.226.254.253', '159.226.254.249'}, {'159.226.254.253', '159.226.254.249'}, {'2001:7c0:2:1090::', '2001:7c0:2:10a8::', '2001:7c0:2:108f::'}, {'67.16.158.41', '67.16.160.177'}, {'2001:1900:108:1::8', '2001:1900:108:2::8'}, {'2001:630:0:10::152', '2001:630:0:10::156'}, {'192.16.166.34', '192.16.166.38'}, {'198.124.80.69', '198.124.80.65'}, {'198.124.80.69', '198.124.80.65'}, {'198.124.80.69', '198.124.80.65'}, {'198.124.80.69', '198.124.80.65'}, {'198.124.80.69', '198.124.80.65'}, {'198.124.80.69', '198.124.80.65'}, {'192.16.166.46', '192.16.166.42'}, {'90.147.80.46', '90.147.80.70', '90.147.80.85'}, {'90.147.80.106', '90.147.80.42'}, {'90.147.80.106', '90.147.80.42'}, {'90.147.80.70', '90.147.80.85'}, {'90.147.80.106', '90.147.80.42'}, {'10.224.2.38', '10.224.2.58'}, {'192.70.69.203', '192.70.69.204'}, {'142.231.1.50', '142.90.1.26'}, {'80.81.192.222', '80.81.193.222'}, {'80.81.192.222', '80.81.193.222'}, {'109.105.97.136', '109

In [11]:
l=len(groups)
for i in range(l):
    for ip in groups[i]:
        for j in range(i+1,l):
            if ip in groups[j]:
                groups[i]=groups[i].union(groups[j])
                groups[j]=set()
                continue

In [12]:
for g in groups:
    if len(g):
        print(g)

{'2001:660:7903:204:2::2', '2001:660:7904:121:2::2'}
{'159.226.254.253', '159.226.254.249'}
{'2001:7c0:2:1090::', '2001:7c0:2:10a8::', '2001:7c0:2:108f::'}
{'67.16.158.41', '67.16.160.177'}
{'2001:1900:108:1::8', '2001:1900:108:2::8'}
{'2001:630:0:10::152', '2001:630:0:10::156'}
{'192.16.166.34', '192.16.166.38'}
{'198.124.80.69', '198.124.80.65'}
{'192.16.166.46', '192.16.166.42'}
{'90.147.80.81', '90.147.80.73', '90.147.80.101', '90.147.80.46', '90.147.80.70', '90.147.80.85'}
{'90.147.80.14', '90.147.80.106', '90.147.80.42', '90.147.80.109', '90.147.80.33', '193.206.128.81', '192.16.166.18'}
{'192.16.166.145', '10.224.2.38', '10.224.2.58'}
{'192.70.69.203', '192.70.69.204'}
{'142.90.1.2', '142.231.1.50', '142.90.1.26', '142.90.1.17'}
{'80.81.192.222', '80.81.193.222'}
{'109.105.97.126', '109.105.97.140', '109.105.97.138', '109.105.97.142', '109.105.97.136', '109.105.97.132', '109.105.98.34'}
{'10.255.222.99', '10.255.221.99'}
{'164.58.244.31', '164.58.244.33'}
{'193.206.128.30', '90.